## Calculating energy cost based on barcode

In [1]:
from skimage import io
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [ ]:
imgtest = io.imread("tumblr_obenth3wAK1qhtovio1_1280.jpeg")

In [ ]:
plt.figure()

In [ ]:
plt.imshow(imgtest)

In [2]:
red_table = [50, 100, 250, 450]
green_table = [50, 100, 250, 450]
blue_table = [100, 200, 500, 950]

'''Calculate power usage of a color value in a pixel'''
def calculate_power_for_color(color_value, power_table):
    if color_value < 60:
       return color_value * power_table[0] / (60 - 0)
    elif color_value < 135:
       return power_table[0] + power_table[1] * (color_value - 60) / (135 - 60)
    elif color_value < 190:
       return power_table[1] + power_table[2] * (color_value - 135) / (190 - 135)
    return power_table[2] + power_table[3] * (color_value - 190) / (255 - 190)

'''Calculate power per pixel'''
def calculate_power(r, g, b):
    return (
        calculate_power_for_color(r, red_table) +
        calculate_power_for_color(g, green_table) +
        calculate_power_for_color(b, blue_table)
    )

In [ ]:
calculate_power(255,255,255)

In [3]:
def calculate_line_power(img):
    line_power = []
    for i in range(img.shape[1]):
        entry = 0
        for j in range(img.shape[0]):
            entry += calculate_power(img[j][i][0], img[j][i][1], img[j][i][2])
        line_power.append(entry)
    return line_power

In [ ]:
# TV specs: https://www.fullspecs.net/smart-tv/lg/oled55bxpua-specifications/
tv_name = "LGOLED55BXPUA"
min_power_consumption = 106
max_power_consumption = 295

# could do a search like: "max power site:www.fullspecs.net/smart-tv/"

In [ ]:
price_per_hour = 10 #10 DKK
movie_length_hours = 2

In [4]:
def calculate_cost(min_power_consumption, max_power_consumption, imgfile, movie_length_hours):
    
    # this part can be calculated offline
    img = io.imread(imgfile)
    line_power = calculate_line_power(img)
    line_power_mean = np.mean(line_power)
    
    
    # this part could be calculated online as we get info about (1) energy prices and (2) tv screen
    return (
        min_power_consumption + 
            (max_power_consumption - min_power_consumption) * line_power_mean
        / (calculate_power(255, 255, 255) * 480)
    ) * movie_length_hours / 1000 * price_per_hour

In [ ]:
calculate_cost(min_power_consumption, max_power_consumption, "tumblr_obenth3wAK1qhtovio1_1280.jpeg", movie_length_hours)


In [ ]:
calculate_cost(min_power_consumption, max_power_consumption, "tumblr_mz3dq6K2wO1qhtovio1_1280.jpeg", movie_length_hours)


In [ ]:
calculate_cost(min_power_consumption, max_power_consumption, "scarface.jpeg", movie_length_hours)

In [ ]:
calculate_cost(min_power_consumption, max_power_consumption, "images/Celebrity (1998).jpg", movie_length_hours)

In [ ]:
calculate_cost(min_power_consumption, max_power_consumption, "images/Clerks II (2006).jpg", movie_length_hours)

In [ ]:
calculate_cost(min_power_consumption, max_power_consumption, "images/The Matrix (1999)\n⇒prints.jpg", movie_length_hours)

## Creating the database of movies with energy cost per pixel

In [5]:
import pandas as pd

In [6]:
movies = pd.read_csv('movies_dedupe.tsv', sep='\t', header=0)
movies = movies.rename(columns={"#title": "title"})
movies

,title,runtime,rating,genre,image_path,imdb_url,soup_url,type
0,Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",./images/Star_Wars:_Episode_VIII_-_The_Last_Je...,https://www.imdb.com/title/tt2527336/,https://moviebarcode.tumblr.com/page/1,movie
1,And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",./images/And_Then_There_Were_None_(1945).jpg,http://www.imdb.com/title/tt0037515/,https://moviebarcode.tumblr.com/page/1,movie
2,Die Ehe der Maria Braun,7200,7.7,Drama,./images/Die_Ehe_der_Maria_Braun__The_Marriage...,http://www.imdb.com/title/tt0079095/,https://moviebarcode.tumblr.com/page/1,movie
3,A Perfect Day,6360,6.8,"Comedy, Drama, War",./images/A_Perfect_Day_(2015).jpg,http://www.imdb.com/title/tt3577624/,https://moviebarcode.tumblr.com/page/1,movie
4,Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",./images/Close_Encounters_of_the_Third_Kind_(1...,http://www.imdb.com/title/tt0075860/,https://moviebarcode.tumblr.com/page/1,movie
...,...,...,...,...,...,...,...,...
1797,127 Hours,5640,7.6,"Biography, Drama",./images/127_Hours_(2010).jpg,http://www.imdb.com/title/tt1542344/,https://moviebarcode.tumblr.com/page/216,movie
1798,The Birds,7140,7.6,"Drama, Horror, Mystery, Romance",./images/The_Birds_(1963).jpg,http://www.imdb.com/title/tt0056869/,https://moviebarcode.tumblr.com/page/216,movie
1799,The King's Speech,7080,8,"Biography, Drama, History",./images/The_King’s_Speech_(2010).jpg,http://www.imdb.com/title/tt1504320/,https://moviebarcode.tumblr.com/page/217,movie
1800,Moon,5820,7.8,"Drama, Mystery, Sci-Fi",./images/Moon_(2009).jpg,http://www.imdb.com/title/tt1182345/,https://moviebarcode.tumblr.com/page/217,movie


In [7]:
movies = movies.drop_duplicates(subset='title', keep="first")
movies

,title,runtime,rating,genre,image_path,imdb_url,soup_url,type
0,Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",./images/Star_Wars:_Episode_VIII_-_The_Last_Je...,https://www.imdb.com/title/tt2527336/,https://moviebarcode.tumblr.com/page/1,movie
1,And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",./images/And_Then_There_Were_None_(1945).jpg,http://www.imdb.com/title/tt0037515/,https://moviebarcode.tumblr.com/page/1,movie
2,Die Ehe der Maria Braun,7200,7.7,Drama,./images/Die_Ehe_der_Maria_Braun__The_Marriage...,http://www.imdb.com/title/tt0079095/,https://moviebarcode.tumblr.com/page/1,movie
3,A Perfect Day,6360,6.8,"Comedy, Drama, War",./images/A_Perfect_Day_(2015).jpg,http://www.imdb.com/title/tt3577624/,https://moviebarcode.tumblr.com/page/1,movie
4,Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",./images/Close_Encounters_of_the_Third_Kind_(1...,http://www.imdb.com/title/tt0075860/,https://moviebarcode.tumblr.com/page/1,movie
...,...,...,...,...,...,...,...,...
1797,127 Hours,5640,7.6,"Biography, Drama",./images/127_Hours_(2010).jpg,http://www.imdb.com/title/tt1542344/,https://moviebarcode.tumblr.com/page/216,movie
1798,The Birds,7140,7.6,"Drama, Horror, Mystery, Romance",./images/The_Birds_(1963).jpg,http://www.imdb.com/title/tt0056869/,https://moviebarcode.tumblr.com/page/216,movie
1799,The King's Speech,7080,8,"Biography, Drama, History",./images/The_King’s_Speech_(2010).jpg,http://www.imdb.com/title/tt1504320/,https://moviebarcode.tumblr.com/page/217,movie
1800,Moon,5820,7.8,"Drama, Mystery, Sci-Fi",./images/Moon_(2009).jpg,http://www.imdb.com/title/tt1182345/,https://moviebarcode.tumblr.com/page/217,movie


In [8]:
movies['runtime_hours'] = movies['runtime']/3600
movies['runtime_hours']

0       2.533333
1       1.616667
2       2.000000
3       1.766667
4       2.300000
          ...   
1797    1.566667
1798    1.983333
1799    1.966667
1800    1.616667
1801    1.550000
Name: runtime_hours, Length: 1802, dtype: float64

In [9]:
movies['pixels_cost'] = 0

In [10]:
movies

,title,runtime,rating,genre,image_path,imdb_url,soup_url,type,runtime_hours,pixels_cost
0,Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",./images/Star_Wars:_Episode_VIII_-_The_Last_Je...,https://www.imdb.com/title/tt2527336/,https://moviebarcode.tumblr.com/page/1,movie,2.533333,0
1,And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",./images/And_Then_There_Were_None_(1945).jpg,http://www.imdb.com/title/tt0037515/,https://moviebarcode.tumblr.com/page/1,movie,1.616667,0
2,Die Ehe der Maria Braun,7200,7.7,Drama,./images/Die_Ehe_der_Maria_Braun__The_Marriage...,http://www.imdb.com/title/tt0079095/,https://moviebarcode.tumblr.com/page/1,movie,2.000000,0
3,A Perfect Day,6360,6.8,"Comedy, Drama, War",./images/A_Perfect_Day_(2015).jpg,http://www.imdb.com/title/tt3577624/,https://moviebarcode.tumblr.com/page/1,movie,1.766667,0
4,Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",./images/Close_Encounters_of_the_Third_Kind_(1...,http://www.imdb.com/title/tt0075860/,https://moviebarcode.tumblr.com/page/1,movie,2.300000,0
...,...,...,...,...,...,...,...,...,...,...
1797,127 Hours,5640,7.6,"Biography, Drama",./images/127_Hours_(2010).jpg,http://www.imdb.com/title/tt1542344/,https://moviebarcode.tumblr.com/page/216,movie,1.566667,0
1798,The Birds,7140,7.6,"Drama, Horror, Mystery, Romance",./images/The_Birds_(1963).jpg,http://www.imdb.com/title/tt0056869/,https://moviebarcode.tumblr.com/page/216,movie,1.983333,0
1799,The King's Speech,7080,8,"Biography, Drama, History",./images/The_King’s_Speech_(2010).jpg,http://www.imdb.com/title/tt1504320/,https://moviebarcode.tumblr.com/page/217,movie,1.966667,0
1800,Moon,5820,7.8,"Drama, Mystery, Sci-Fi",./images/Moon_(2009).jpg,http://www.imdb.com/title/tt1182345/,https://moviebarcode.tumblr.com/page/217,movie,1.616667,0


In [11]:
line_powers = []

error_images_index = []

for i in range(0, len(movies)):
    print(movies.iloc[i].image_path)
    img = io.imread(movies.iloc[i].image_path)
    if img[0].shape == (1280, 3):
        line_power = calculate_line_power(img)
        line_power_mean = np.mean(line_power)
        line_powers.append(line_power_mean)
        movies['pixels_cost'][i] = line_power_mean
    else:
        error_images_index.append(i)
        

./images/La_jetée_(1962).jpg
./images/Animal_Farm_(1954).jpg


/var/folders/c3/y1jl2pxx1011s3xbcm4ls1400000gn/T/ipykernel_7596/3028741032.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['pixels_cost'][i] = line_power_mean


./images/Sequence_from_Night_on_Earth_(1991).jpg
./images/Fresh_Guacamole_(2012).jpg
./images/The_Ghost_and_the_Darkness_(1996).jpg
./images/Poulet_aux_prunes__Chicken_with_Plums_(2011).jpg
./images/That_Thing_You_Do!_(1996).jpg
./images/Discopathe_(2013).jpg
./images/A_Nightmare_on_Elm_Street_(1984).jpg
./images/House_on_Haunted_Hill_(1959).jpg
./images/Don’t_Hug_Me_I’m_Scared_2:_Time_(2014).jpg
./images/The_Two_Faces_of_January_(2014).jpg
./images/The_Signal_(2014).jpg
./images/Bal__Honey_(2010).jpg
./images/The_Dogs_of_War_(1980).jpg
./images/Castello_Cavalcanti_(2013).jpg
./images/L'amour_est_un_crime_parfait__Love_Is_the_Perfect_Crime_(2013).jpg
./images/Sequence_from_Black_Swan_(2010).jpg
./images/Before_Night_Falls_(2000).jpg
./images/The_Fugitive_(1993).jpg
./images/Sequence_from_It’s_Always_Fair_Weather_(1955).jpg
./images/Captain_America:_The_Winter_Soldier_(2014).jpg
./images/Into_the_Abyss_(2011).jpg
./images/Sade_(2000).jpg
./images/Juggernaut_(1974).jpg
./images/Grace_of_

./images/The_Hunger_Games_(2012).jpg
./images/Small_Time_Crooks_(2000).jpg
./images/Ivanovo_detstvo__Ivan’s_Childhood_(1962).jpg
./images/Notes_on_a_Scandal_(2006).jpg
./images/The_Bourne_Legacy_(2012).jpg
./images/Johann_Mouse_(1952).jpg
./images/Le_refuge__The_Refuge_(2009).jpg
./images/Mon_oncle_(1958).jpg
./images/Memoirs_of_a_Geisha_(2005).jpg
./images/Aningaaq_(2013).jpg
./images/La_battaglia_di_Algeri__The_Battle_of_Algiers_(1966).jpg
./images/Offret__The_Sacrifice_(1986).jpg
./images/Tweetie_Pie_(1947).jpg
./images/Playtime_(1967).jpg
./images/Sweet_and_Lowdown_(1999).jpg
./images/Prometheus_(2012).jpg
./images/Eldfjall__Volcano_(2011).jpg
./images/Nebraska_(2013).jpg
./images/Dallas_Buyers_Club_(2013).jpg
./images/The_Bourne_Supremacy_(2004).jpg
./images/The_Little_Orphan_(1948).jpg
./images/Desk_Set_(1957).jpg
./images/The_Great_Escape_(1963).jpg
./images/Mighty_Aphrodite_(1995).jpg
./images/Invasion_of_the_Body_Snatchers_(1956).jpg
./images/Elephant_(2003).jpg
./images/Delic

./images/Moneyball_(2011).jpg
./images/Il_gattopardo__The_Leopard_(1963).jpg
./images/Die_Wand__The_Wall_(2012).jpg
./images/Sequence_from_Django_Unchained_(2012).jpg
./images/Cloud_Atlas_(2012).jpg
./images/The_Influence_of_Ocular_Light_Perception_on_Metabolism_in_Man_and_in_Animal_(2005).jpg
./images/Sleepless_in_Seattle_(1993).jpg
./images/Goodbye_Again_(1961).jpg
./images/Help!_(1965).jpg
./images/Sequence_from_Antichrist_(2009).jpg
./images/James_Bond_-_GoldenEye_(1995).jpg
./images/Sequence_from_American_Psycho_(2000).jpg
./images/Hansel_&_Gretel:_Witch_Hunters_(2013).jpg
./images/Au_hasard_Balthazar_(1966).jpg
./images/Niagara_(1953).jpg
./images/Gangster_Squad_(2013).jpg
./images/Ali_(2001).jpg
./images/Sequence_from_The_Darjeeling_Limited_(2007).jpg
./images/Avatar_(2009).jpg
./images/Exotica_(1994).jpg
./images/James_Bond_-_The_Living_Daylights_(1987).jpg
./images/Sequence_from_The_Shining_(1980).jpg
./images/Spaceballs_(1987).jpg
./images/Hiroshima,_mon_amour_(1959).jpg
./im

./images/Gremlins_(1984).jpg
./images/Désirée_(1954).jpg
./images/Pixar_Shorts:_Luxo_Jr._(1986).jpg
./images/The_Avengers_(2012).jpg
./images/The_Bad_Lieutenant:_Port_of_Call_-_New_Orleans_(2009).jpg
./images/Gorky_Park_(1983).jpg
./images/The_Thing_(2011).jpg
./images/La_piscine__The_Swimming_Pool_(1969).jpg
./images/Koyaanisqatsi_(1982).jpg
./images/Pixar_Shorts:_The_Adventures_of_André_and_Wally_B._(1984).jpg
./images/Buena_Vista_Social_Club_(1999).jpg
./images/Closer_(2004).jpg
./images/High_Fidelity_(2000).jpg
./images/The_Ides_of_March_(2011).jpg
./images/Ikiru_(1952).jpg
./images/Brave_(2012).jpg
./images/Iron_Sky_(2012).jpg
./images/Chinatown_(1974).jpg
./images/The_Expendables_2_(2012).jpg
./images/The_Expendables_(2010).jpg
./images/Inauguration_of_the_Pleasure_Dome_(1954).jpg
./images/Grumpier_Old_Men_(1995).jpg
./images/Eaux_d'artifice_(1953).jpg
./images/Persona_(1966).jpg
./images/Sequence_from_This_Is_Spinal_Tap_(1984).jpg
./images/A_Big_Grey-Blue_Bird__Ein_großer_graubl

./images/Santa_sangre_(1989).jpg
./images/The_Girl_with_the_Dragon_Tattoo_(2011).jpg
./images/The_Girl_with_the_Dragon_Tattoo__Män_som_hatar_kvinnor_(2009).jpg
./images/Ronja_Rövardotter__Ronia,_the_Robber’s_Daughter_(1984).jpg
./images/The_Asphalt_Jungle_(1950).jpg
./images/Le_charme_discret_de_la_bourgeoisie_(1972).jpg
./images/The_Station_Agent_(2003).jpg
./images/My_Fair_Lady_(1964).jpg
./images/Workingman’s_Death_(2005).jpg
./images/Le_conseguenze_dell'amore__The_Consequences_of_Love_(2004).jpg
./images/Sequence_from_Toy_Story_3_(2010).jpg
./images/Manhattan_(1979).jpg
./images/Sequence_from_Manufactured_Landscapes_(2006).jpg
./images/Autumn_Milk__Herbstmilch_(1989).jpg
./images/Fantastic_Planet__La_planète_sauvage_(1973).jpg
./images/Foxy_Brown_(1974).jpg
./images/Jackie_Brown_(1997).jpg
./images/Captain_America:_The_First_Avenger_(2011).jpg
./images/Ship_of_Fools_(1965).jpg
./images/The_Adventures_of_Tintin_(2011).jpg
./images/Sequence_from_Up_(2009).jpg
./images/Europa_(1991).j

./images/My_Sassy_Girl__Yeopgijeogin_geunyeo_(2001).jpg
./images/Annie_Hall_(1977).jpg
./images/Chungking_Express__Chung_Hing_sam_lam_(1994).jpg
./images/The_Nightmare_Before_Christmas_(1993).jpg
./images/The_Son’s_Room__La_stanza_del_figlio_(2001).jpg
./images/Hobo_with_a_Shotgun_(2011).jpg
./images/Nothing_(2003).jpg
./images/X-Men_(2000).jpg
./images/Taxidermia_(2006).jpg
./images/500_Days_of_Summer_(2009).jpg
./images/THX_1138_(1971).jpg
./images/Madagascar:_Escape_2_Africa_(2008).jpg
./images/The_Mourning_Forest__Mogari_no_mori_(2007).jpg
./images/Minority_Report_(2002).jpg
./images/Reservoir_Dogs_(1992).jpg
./images/I’m_a_Cyborg,_But_That’s_OK__Ssaibogeujiman_gwaenchanha_(2006).jpg
./images/The_English_Patient_(1996).jpg
./images/Willy_Wonka_&_the_Chocolate_Factory_(1971).jpg
./images/Dr._Strangelove_or:_How_I_Learned_to_Stop_Worrying_and_Love_the_Bomb_(1964).jpg
./images/Marie_Antoinette_(2006).jpg
./images/Happy-End_(1996).jpg
./images/Monty_Python_and_the_Holy_Grail_(1975).jpg

./images/Predators_(2010).jpg
./images/Predator_2_(1990).jpg
./images/Predator_(1987).jpg
./images/Babel_(2006).jpg
./images/The_War_of_the_Worlds_(1953).jpg
./images/A_Single_Man_(2009).jpg
./images/Snow_White_and_the_Seven_Dwarfs_(1937).jpg
./images/Gangs_of_New_York_(2002).jpg
./images/American_Beauty_(1999).jpg
./images/Bambi_(1942).jpg
./images/Rear_Window_(1954).jpg
./images/Slumdog_Millionaire_(2008).jpg
./images/Scarface_(1983).jpg
./images/In_the_Mood_for_Love__Fa_yeung_nin_wa_(2000).jpg
./images/The_Proposition_(2005).jpg
./images/Midnight_Cowboy_(1969).jpg
./images/Æon_Flux_(2005).jpg
./images/Goodfellas_(1990).jpg
./images/Raging_Bull_(1980).jpg
./images/Taxi_Driver_(1976).jpg
./images/Requiem_for_a_Dream_(2000).jpg
./images/Astro_Boy_(2009).jpg
./images/The_Kids_Are_All_Right_(2010).jpg
./images/The_Social_Network_(2010).jpg
./images/How_to_Train_Your_Dragon_(2010).jpg
./images/Winter’s_Bone_(2010).jpg
./images/Pleasantville_(1998).jpg
./images/City_of_God__Cidade_de_Deus_

In [12]:
error_images_index

[545, 578, 650, 664, 1092, 1209, 1213, 1222, 1229, 1244, 1329, 1724]

In [ ]:
"""
# do this here just for testing, otherwise to be done in extension

def calculate_movie_cost(min_power_consumption, max_power_consumption, pixels_cost, movie_length_hours, price_per_hour):
    return (
        min_power_consumption + 
            (max_power_consumption - min_power_consumption) * pixels_cost
        / (calculate_power(255, 255, 255) * 480)
    ) * movie_length_hours / 1000 * price_per_hour

movies_cost = []

for i in range(0,len(movies)):
    movies_cost.append(calculate_movie_cost(min_power_consumption=106, 
                         max_power_consumption=295, 
                         pixels_cost=movies.iloc[i].pixels_cost, 
                         movie_length_hours=movies.iloc[i].runtime_hours,
                         price_per_hour=10))
    
movies['movie_cost'] = movies_cost
movies
"""

In [13]:
movies

,title,runtime,rating,genre,image_path,imdb_url,soup_url,type,runtime_hours,pixels_cost
0,Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",./images/Star_Wars:_Episode_VIII_-_The_Last_Je...,https://www.imdb.com/title/tt2527336/,https://moviebarcode.tumblr.com/page/1,movie,2.533333,0.000000
1,And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",./images/And_Then_There_Were_None_(1945).jpg,http://www.imdb.com/title/tt0037515/,https://moviebarcode.tumblr.com/page/1,movie,1.616667,0.000000
2,Die Ehe der Maria Braun,7200,7.7,Drama,./images/Die_Ehe_der_Maria_Braun__The_Marriage...,http://www.imdb.com/title/tt0079095/,https://moviebarcode.tumblr.com/page/1,movie,2.000000,0.000000
3,A Perfect Day,6360,6.8,"Comedy, Drama, War",./images/A_Perfect_Day_(2015).jpg,http://www.imdb.com/title/tt3577624/,https://moviebarcode.tumblr.com/page/1,movie,1.766667,0.000000
4,Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",./images/Close_Encounters_of_the_Third_Kind_(1...,http://www.imdb.com/title/tt0075860/,https://moviebarcode.tumblr.com/page/1,movie,2.300000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1797,127 Hours,5640,7.6,"Biography, Drama",./images/127_Hours_(2010).jpg,http://www.imdb.com/title/tt1542344/,https://moviebarcode.tumblr.com/page/216,movie,1.566667,123256.765701
1798,The Birds,7140,7.6,"Drama, Horror, Mystery, Romance",./images/The_Birds_(1963).jpg,http://www.imdb.com/title/tt0056869/,https://moviebarcode.tumblr.com/page/216,movie,1.983333,121872.588420
1799,The King's Speech,7080,8,"Biography, Drama, History",./images/The_King’s_Speech_(2010).jpg,http://www.imdb.com/title/tt1504320/,https://moviebarcode.tumblr.com/page/217,movie,1.966667,58464.097579
1800,Moon,5820,7.8,"Drama, Mystery, Sci-Fi",./images/Moon_(2009).jpg,http://www.imdb.com/title/tt1182345/,https://moviebarcode.tumblr.com/page/217,movie,1.616667,162755.293583


In [15]:
movies = movies.set_index(movies.columns[0])
movies

,runtime,rating,genre,image_path,imdb_url,soup_url,type,runtime_hours,pixels_cost
title,,,,,,,,,
Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",./images/Star_Wars:_Episode_VIII_-_The_Last_Je...,https://www.imdb.com/title/tt2527336/,https://moviebarcode.tumblr.com/page/1,movie,2.533333,0.000000
And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",./images/And_Then_There_Were_None_(1945).jpg,http://www.imdb.com/title/tt0037515/,https://moviebarcode.tumblr.com/page/1,movie,1.616667,0.000000
Die Ehe der Maria Braun,7200,7.7,Drama,./images/Die_Ehe_der_Maria_Braun__The_Marriage...,http://www.imdb.com/title/tt0079095/,https://moviebarcode.tumblr.com/page/1,movie,2.000000,0.000000
A Perfect Day,6360,6.8,"Comedy, Drama, War",./images/A_Perfect_Day_(2015).jpg,http://www.imdb.com/title/tt3577624/,https://moviebarcode.tumblr.com/page/1,movie,1.766667,0.000000
Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",./images/Close_Encounters_of_the_Third_Kind_(1...,http://www.imdb.com/title/tt0075860/,https://moviebarcode.tumblr.com/page/1,movie,2.300000,0.000000
...,...,...,...,...,...,...,...,...,...
127 Hours,5640,7.6,"Biography, Drama",./images/127_Hours_(2010).jpg,http://www.imdb.com/title/tt1542344/,https://moviebarcode.tumblr.com/page/216,movie,1.566667,123256.765701
The Birds,7140,7.6,"Drama, Horror, Mystery, Romance",./images/The_Birds_(1963).jpg,http://www.imdb.com/title/tt0056869/,https://moviebarcode.tumblr.com/page/216,movie,1.983333,121872.588420
The King's Speech,7080,8,"Biography, Drama, History",./images/The_King’s_Speech_(2010).jpg,http://www.imdb.com/title/tt1504320/,https://moviebarcode.tumblr.com/page/217,movie,1.966667,58464.097579


In [16]:
movies = movies.drop(columns=['image_path', 'soup_url'])
movies

,runtime,rating,genre,imdb_url,type,runtime_hours,pixels_cost
title,,,,,,,
Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",https://www.imdb.com/title/tt2527336/,movie,2.533333,0.000000
And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",http://www.imdb.com/title/tt0037515/,movie,1.616667,0.000000
Die Ehe der Maria Braun,7200,7.7,Drama,http://www.imdb.com/title/tt0079095/,movie,2.000000,0.000000
A Perfect Day,6360,6.8,"Comedy, Drama, War",http://www.imdb.com/title/tt3577624/,movie,1.766667,0.000000
Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",http://www.imdb.com/title/tt0075860/,movie,2.300000,0.000000
...,...,...,...,...,...,...,...
127 Hours,5640,7.6,"Biography, Drama",http://www.imdb.com/title/tt1542344/,movie,1.566667,123256.765701
The Birds,7140,7.6,"Drama, Horror, Mystery, Romance",http://www.imdb.com/title/tt0056869/,movie,1.983333,121872.588420
The King's Speech,7080,8,"Biography, Drama, History",http://www.imdb.com/title/tt1504320/,movie,1.966667,58464.097579


In [ ]:
movies.sort_values(by=['pixels_cost'])

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
fig, ax = plt.subplots()
ax.scatter(movies['rating'], movies['pixels_cost'])
for i, txt in enumerate(movies.index):
    ax.annotate(txt, (movies['rating'][i], movies['pixels_cost'][i]))
ax.set_xlabel("IMDb Rating")
ax.set_ylabel("Pixels Cost")

In [20]:
part_1 = pd.read_json('movies.json', orient="index")

In [21]:
part_1

,runtime,rating,genre,imdb_url,type,runtime_hours,pixels_cost
Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",https://www.imdb.com/title/tt2527336/,movie,2.533333,92518.447508
And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",http://www.imdb.com/title/tt0037515/,movie,1.616667,86477.686402
Die Ehe der Maria Braun,7200,7.7,Drama,http://www.imdb.com/title/tt0079095/,movie,2.000000,88595.179142
A Perfect Day,6360,6.8,"Comedy, Drama, War",http://www.imdb.com/title/tt3577624/,movie,1.766667,238280.291336
Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",http://www.imdb.com/title/tt0075860/,movie,2.300000,133868.685903
...,...,...,...,...,...,...,...
Deep Blue Sea,6300,5.9,"Action, Adventure, Sci-Fi, Thriller",http://www.imdb.com/title/tt0149261/,movie,1.750000,97326.968572
Nuovomondo,7080,6.8,"Drama, History, Romance",http://www.imdb.com/title/tt0465188/,movie,1.966667,100573.648358
How to Train Your Dragon 2,6120,7.8,"Animation, Action, Adventure, Comedy, Family, ...",http://www.imdb.com/title/tt1646971/,movie,1.700000,186568.817933
Point Blank,5520,7.3,"Crime, Drama, Thriller",http://www.imdb.com/title/tt0062138/,movie,1.533333,95951.170892


In [23]:
for i in range(0, 545):
    movies['pixels_cost'][i] = part_1['pixels_cost'][i]

/var/folders/c3/y1jl2pxx1011s3xbcm4ls1400000gn/T/ipykernel_7596/68575987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['pixels_cost'][i] = part_1['pixels_cost'][i]


In [31]:
movies

,runtime,rating,genre,imdb_url,type,runtime_hours,pixels_cost
title,,,,,,,
Star Wars: Episode VIII - The Last Jedi,9120,6.9,"Action, Adventure, Fantasy, Sci-Fi",https://www.imdb.com/title/tt2527336/,movie,2.533333,92518.447508
And Then There Were None,5820,7.4,"Crime, Drama, Mystery, Thriller",http://www.imdb.com/title/tt0037515/,movie,1.616667,86477.686402
Die Ehe der Maria Braun,7200,7.7,Drama,http://www.imdb.com/title/tt0079095/,movie,2.000000,88595.179142
A Perfect Day,6360,6.8,"Comedy, Drama, War",http://www.imdb.com/title/tt3577624/,movie,1.766667,238280.291336
Close Encounters of the Third Kind,8280,7.6,"Drama, Sci-Fi",http://www.imdb.com/title/tt0075860/,movie,2.300000,133868.685903
...,...,...,...,...,...,...,...
127 Hours,5640,7.6,"Biography, Drama",http://www.imdb.com/title/tt1542344/,movie,1.566667,123256.765701
The Birds,7140,7.6,"Drama, Horror, Mystery, Romance",http://www.imdb.com/title/tt0056869/,movie,1.983333,121872.588420
The King's Speech,7080,8,"Biography, Drama, History",http://www.imdb.com/title/tt1504320/,movie,1.966667,58464.097579


In [30]:
movies = movies[movies['pixels_cost']!=0]

In [32]:
with open('movies_scored.json', 'w') as f:
    f.write(movies.to_json(orient="index"))

In [33]:
movies.to_csv('movies_scored.tsv', sep = "\t")